In [732]:
import pandas as pd

In [733]:
train=pd.read_csv('./train_df_errno.csv')
test=pd.read_csv('./test_df.csv')
sub=pd.read_csv('./sample_submission.csv')
age=pd.read_csv('./age_gender_info.csv')
train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1008, 14), (150, 2), (16, 23))

In [734]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        2896 non-null   object 
 1   총세대수        2896 non-null   int64  
 2   임대건물구분      2896 non-null   object 
 3   지역          2896 non-null   object 
 4   공급유형        2896 non-null   object 
 5   전용면적        2896 non-null   float64
 6   전용면적별세대수    2896 non-null   int64  
 7   공가수         2896 non-null   float64
 8   자격유형        2896 non-null   object 
 9   임대보증금       2327 non-null   object 
 10  임대료         2327 non-null   object 
 11  10분내지하철수    2685 non-null   float64
 12  10분내버스정류장수  2892 non-null   float64
 13  단지내주차면수     2896 non-null   float64
 14  등록차량수       2896 non-null   float64
dtypes: float64(6), int64(2), object(7)
memory usage: 339.5+ KB


In [735]:
train.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수      211
10분내버스정류장수      4
단지내주차면수         0
등록차량수           0
dtype: int64

In [736]:
train['임대료'].value_counts()

-         21
390000    19
79980     18
100920    16
167230    15
          ..
82940      1
133570     1
103840     1
273260     1
236220     1
Name: 임대료, Length: 975, dtype: int64

In [737]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')

### 결측치

In [738]:
test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         180
임대료           180
10분내지하철수       38
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

In [739]:
train.loc[train['임대보증금'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
80,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
81,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
82,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
83,C1925,601,상가,강원도,임대상가,72.16,1,9.0,D,NaN,NaN,0.0,4.0,117.0,75.0
93,C1874,619,상가,충청남도,임대상가,12.62,1,2.0,D,NaN,NaN,NaN,2.0,97.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2311,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2312,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2313,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2314,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0


### 1.자격유형: 결측치 채우고, mapping

In [740]:
test.loc[test['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [741]:
grouped=test.groupby(['단지코드','임대건물구분','지역','공급유형'])
group1=grouped.get_group(('C2411','아파트','경상남도','국민임대'))
group2=grouped.get_group(('C2253','아파트','강원도','영구임대'))
print(group1)
print(group2)

      단지코드  총세대수 임대건물구분    지역  공급유형   전용면적  전용면적별세대수   공가수 자격유형     임대보증금  \
193  C2411   962    아파트  경상남도  국민임대  39.43        56  25.0    A  11992000   
194  C2411   962    아파트  경상남도  국민임대  39.72       336  25.0    A  11992000   
195  C2411   962    아파트  경상남도  국민임대  39.82       179  25.0    A  11992000   
196  C2411   962    아파트  경상남도  국민임대  46.90       240  25.0  NaN  71950000   
197  C2411   962    아파트  경상남도  국민임대  51.93       150  25.0    A  21586000   

        임대료  10분내지하철수  10분내버스정류장수  단지내주차면수  
193  100720       0.0         2.0    840.0  
194  100720       0.0         2.0    840.0  
195  100720       0.0         2.0    840.0  
196   37470       0.0         2.0    840.0  
197  171480       0.0         2.0    840.0  
      단지코드  총세대수 임대건물구분   지역  공급유형   전용면적  전용면적별세대수  공가수 자격유형    임대보증금    임대료  \
258  C2253  1161    아파트  강원도  영구임대  26.37       745  0.0  NaN  2249000  44770   
259  C2253  1161    아파트  강원도  영구임대  31.32       239  0.0    C  3731000  83020   
260  C2253  1161    아파트 

In [742]:
test.loc[196,'자격유형']='A'
test.loc[258,'자격유형']='C'

In [743]:
test['자격유형'].isnull().sum()

0

In [744]:
train['자격유형'].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O'], dtype=object)

In [745]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5,
            'F':6, 'G':7, 'H':8, 'I':9, 'J':10,
            'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }
train['자격유형']=train['자격유형'].map(mapping).astype(int)
test['자격유형']=test['자격유형'].map(mapping).astype(int)

### 2.도보 10분거리 내 버스정류장 수: 결측치 채우고, qcut

In [746]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [747]:
train.loc[train['10분내버스정류장수'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2293,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,1,46000000,456000,NaN,NaN,1066.0,1214.0
2294,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,1,57000000,462000,NaN,NaN,1066.0,1214.0
2295,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,1,57000000,462000,NaN,NaN,1066.0,1214.0
2296,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,1,57000000,462000,NaN,NaN,1066.0,1214.0


In [748]:
grouped=test.groupby(['임대건물구분','지역','공급유형','자격유형'])
group1=grouped.get_group(('아파트','경상남도','공공임대(10년)',1))
group1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
810,C2175,595,아파트,경상남도,공공임대(10년),74.82,118,5.0,1,47000000,470000,0.0,3.0,685.0
811,C2175,595,아파트,경상남도,공공임대(10년),84.69,297,5.0,1,58000000,520000,0.0,3.0,685.0
812,C2175,595,아파트,경상남도,공공임대(10년),84.86,143,5.0,1,58000000,520000,0.0,3.0,685.0
813,C2175,595,아파트,경상남도,공공임대(10년),84.89,13,5.0,1,58000000,520000,0.0,3.0,685.0
814,C2175,595,아파트,경상남도,공공임대(10년),84.94,13,5.0,1,58000000,520000,0.0,3.0,685.0
815,C2175,595,아파트,경상남도,공공임대(10년),84.94,11,5.0,1,58000000,520000,0.0,3.0,685.0


In [749]:
train.loc[train['10분내버스정류장수'].isnull(),"10분내버스정류장수"]=5
train.loc[train['10분내버스정류장수'].isnull(),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


In [750]:
train['10분내버스정류장수'].value_counts()

2.0     776
3.0     737
4.0     485
1.0     270
5.0     166
6.0     153
8.0      99
10.0     80
7.0      65
12.0     22
20.0     15
15.0     11
16.0      6
19.0      4
11.0      3
14.0      2
0.0       2
Name: 10분내버스정류장수, dtype: int64

In [751]:
train['label_bus']=pd.qcut(train['10분내버스정류장수'],20,labels=False,duplicates='drop')

In [752]:
test['label_bus']=pd.qcut(test['10분내버스정류장수'],20,labels=False,duplicates='drop')

### 3.도보 10분거리 내 지하철역 수(환승노선 수 반영):결측치 채우고, qcut

In [753]:
train['10분내지하철수'].isnull().sum()

211

In [754]:
test['10분내지하철수'].isnull().sum()

38

In [755]:
import numpy as np

In [756]:
train.loc[train['10분내지하철수'].isnull(),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,label_bus
86,C1312,518,아파트,충청남도,국민임대,39.72,60,12.0,1,17460000,122210,NaN,3.0,527.0,359.0,2
87,C1312,518,아파트,충청남도,국민임대,39.98,89,12.0,1,17460000,122210,NaN,3.0,527.0,359.0,2
88,C1312,518,아파트,충청남도,국민임대,41.55,225,12.0,1,19954000,130940,NaN,3.0,527.0,359.0,2
89,C1312,518,아파트,충청남도,국민임대,46.90,143,12.0,1,28687000,149660,NaN,3.0,527.0,359.0,2
90,C1874,619,아파트,충청남도,영구임대,26.37,294,2.0,3,3141000,69900,NaN,2.0,97.0,62.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,C1350,1401,아파트,대전광역시,공공임대(10년),59.91,13,2.0,1,37474000,519350,NaN,6.0,1636.0,2315.0,5
2318,C1350,1401,아파트,대전광역시,공공임대(10년),59.92,223,2.0,1,37385000,516130,NaN,6.0,1636.0,2315.0,5
2319,C1350,1401,아파트,대전광역시,공공임대(분납),51.99,146,2.0,1,54612000,131130,NaN,6.0,1636.0,2315.0,5
2320,C1350,1401,아파트,대전광역시,공공임대(분납),59.91,32,2.0,1,63585000,152680,NaN,6.0,1636.0,2315.0,5


In [757]:
#10분내버스정류장 수에 비례해서 넣을까?
#train.sort_values('10분내지하철수')['10분내버스정류장수']

In [758]:
#최빈값으로 넣을까?
test['10분내지하철수'].mode()

0    0.0
dtype: float64

In [759]:
train['10분내지하철수']=train['10분내지하철수'].fillna(train.groupby('10분내버스정류장수')['10분내지하철수'].transform(np.mean))

In [760]:
test['10분내지하철수']=test['10분내지하철수'].fillna(test.groupby('10분내버스정류장수')['10분내지하철수'].transform(np.mean))

In [761]:
test.loc[test['10분내지하철수'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,label_bus


In [762]:
test['10분내지하철수']=test['10분내버스정류장수'].fillna(0)

In [763]:
train['label_sub']=pd.qcut(train['10분내지하철수'],20,labels=False,duplicates='drop')

In [764]:
test['label_sub']=pd.qcut(test['10분내지하철수'],20,labels=False,duplicates='drop')

### 4.단지코드: 결측치 처리

In [765]:
test.loc[test['단지코드'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,label_bus,label_sub


In [766]:
grouped=train.groupby(['임대건물구분','지역','공급유형'])
group2=grouped.get_group(('아파트','대전광역시','행복주택'))
group2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,label_bus,label_sub
2790,C2298,581,아파트,대전광역시,행복주택,16.85,96,21.0,10,16400000,79000,0.0,3.0,408.0,444.0,2,0
2791,C2298,581,아파트,대전광역시,행복주택,26.64,204,21.0,10,24520000,118000,0.0,3.0,408.0,444.0,2,0
2792,C2298,581,아파트,대전광역시,행복주택,26.64,30,21.0,10,24520000,118000,0.0,3.0,408.0,444.0,2,0
2793,C2298,581,아파트,대전광역시,행복주택,36.86,148,21.0,10,33640000,162000,0.0,3.0,408.0,444.0,2,0
2794,C2298,581,아파트,대전광역시,행복주택,36.86,100,21.0,10,33640000,162000,0.0,3.0,408.0,444.0,2,0


In [767]:
test[test['단지코드']=='C2298']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,label_bus,label_sub


In [768]:
test['단지코드']=test['단지코드'].fillna('C2298')

In [769]:
test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         180
임대료           180
10분내지하철수        0
10분내버스정류장수      0
단지내주차면수         0
label_bus       0
label_sub       0
dtype: int64

In [770]:
train.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수        0
10분내버스정류장수      0
단지내주차면수         0
등록차량수           0
label_bus       0
label_sub       0
dtype: int64

### 5. 공급유형_mapping

In [771]:
train['공급유형'].value_counts()

국민임대         1736
임대상가          562
행복주택          202
공공임대(10년)     182
영구임대          152
공공임대(50년)      31
공공임대(분납)       12
장기전세            9
공공분양            7
공공임대(5년)        3
Name: 공급유형, dtype: int64

In [772]:
mapping = { '국민임대':1, '임대상가':2, '행복주택':3, '공공임대(10년)':4, 
           '영구임대':5,'공공임대(50년)':6,
           '공공임대(분납)':7, '장기전세':8, '공공분양':9, '공공임대(5년)':10 }
train['공급유형'] =train['공급유형'].map(mapping).astype(int)
test['공급유형'] =test['공급유형'].map(mapping).astype(int)

### 6. 전용면적별 세대수_qcut

In [773]:
train['전용면적별세대수'].value_counts()

1       562
30       49
60       48
24       45
90       42
       ... 
606       1
616       1
214       1
620       1
1865      1
Name: 전용면적별세대수, Length: 400, dtype: int64

In [774]:
train['label_generation_num']=pd.qcut(train['전용면적별세대수'],20,labels=False,duplicates='drop')

In [775]:
test['label_generation_num']=pd.qcut(test['전용면적별세대수'],20,labels=False,duplicates='drop')

### 7. 지역_mapping

In [776]:
train['지역'].value_counts()

경기도        602
경상남도       369
부산광역시      328
대전광역시      266
강원도        187
충청북도       163
충청남도       158
전라남도       147
광주광역시      141
경상북도       111
전라북도       109
대구광역시      102
제주특별자치도     91
서울특별시       64
울산광역시       34
세종특별자치시     24
Name: 지역, dtype: int64

In [777]:
mapping1 = { '경기도':1, '경상남도':2, '부산광역시':3, '대전광역시':4, 
           '강원도':5,'충청북도':6,'충청남도':7, '전라남도':8, '광주광역시':9, 
            '경상북도':10, '전라북도':11,'대구광역시':12,'제주특별자치도':13,'서울특별시':14,'세종특별자치시':15,'울산광역시':16 }
train['지역'] =train['지역'].map(mapping1).astype(int)
test['지역'] =test['지역'].map(mapping1).astype(int)

### 8. 임대건물구분 mapping

In [778]:
train['임대건물구분'].unique()

array(['아파트', '상가'], dtype=object)

In [779]:
mapping1 = { '아파트':1, '상가':2}
train['임대건물구분'] =train['임대건물구분'].map(mapping1).astype(int)
test['임대건물구분'] =test['임대건물구분'].map(mapping1).astype(int)

### 9.공가수 qcut

In [780]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,label_bus,label_sub,label_generation_num
0,C2515,545,1,2,1,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,205.0,2,0,15
1,C2515,545,1,2,1,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,205.0,2,0,6
2,C2515,545,1,2,1,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,205.0,2,0,2
3,C2515,545,1,2,1,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,205.0,2,0,4
4,C2515,545,1,2,1,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,205.0,2,0,2


In [781]:
train['공가수'].value_counts()

2.0     254
1.0     245
7.0     230
13.0    229
3.0     110
4.0     107
10.0    106
9.0     104
21.0    104
0.0     101
15.0    100
16.0     96
14.0     84
29.0     81
5.0      78
17.0     70
8.0      58
25.0     56
22.0     54
28.0     49
20.0     48
26.0     47
19.0     47
6.0      44
23.0     37
12.0     36
32.0     35
11.0     34
18.0     32
24.0     31
34.0     24
27.0     20
30.0     19
36.0     18
46.0     14
31.0     13
33.0     12
40.0     11
49.0      9
55.0      8
47.0      8
39.0      8
43.0      5
38.0      5
35.0      4
42.0      4
41.0      4
37.0      3
Name: 공가수, dtype: int64

In [782]:
train['공가수']=pd.qcut(train['공가수'],20,labels=False,duplicates='drop')
test['공가수']=pd.qcut(test['공가수'],20,labels=False,duplicates='drop')

### 머신러닝

In [783]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [784]:
### 한글 폰트 설정
import matplotlib
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform
path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")
matplotlib.rcParams['axes.unicode_minus'] = False

In [785]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [786]:
train.shape

(2896, 18)

In [787]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수',
       'label_bus', 'label_sub', 'label_generation_num'],
      dtype='object')

In [788]:
#sel=['label_generation_num','label_bus','단지내주차면수','공급유형','총세대수','전용면적','공가수','지역']
sel=['label_generation_num','label_bus','단지내주차면수','공급유형','총세대수','전용면적','공가수','지역','임대건물구분','자격유형']
X=train[sel]
y=train['등록차량수']

In [808]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,random_state=42)
model = RandomForestClassifier(n_estimators=8, random_state=0)  # 5개의 트리
model.fit(X_train, y_train)
print("학습용 세트 정확도 : {:.3f}".format(model.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(model.score(X_test, y_test)))

학습용 세트 정확도 : 0.999
테스트 세트 정확도 : 0.971


In [809]:
model.estimators_

[DecisionTreeClassifier(max_features='auto', random_state=209652396),
 DecisionTreeClassifier(max_features='auto', random_state=398764591),
 DecisionTreeClassifier(max_features='auto', random_state=924231285),
 DecisionTreeClassifier(max_features='auto', random_state=1478610112),
 DecisionTreeClassifier(max_features='auto', random_state=441365315),
 DecisionTreeClassifier(max_features='auto', random_state=1537364731),
 DecisionTreeClassifier(max_features='auto', random_state=192771779),
 DecisionTreeClassifier(max_features='auto', random_state=1491434855)]

In [810]:
for one_model in model.estimators_:
    print("학습용 세트 정확도 : {:.3f}".format(one_model.score(X_train, y_train)))
    print("테스트 세트 정확도 : {:.3f}".format(one_model.score(X_test, y_test)))
    print()

학습용 세트 정확도 : 0.000
테스트 세트 정확도 : 0.000

학습용 세트 정확도 : 0.000
테스트 세트 정확도 : 0.000

학습용 세트 정확도 : 0.000
테스트 세트 정확도 : 0.000

학습용 세트 정확도 : 0.000
테스트 세트 정확도 : 0.006

학습용 세트 정확도 : 0.000
테스트 세트 정확도 : 0.000

학습용 세트 정확도 : 0.000
테스트 세트 정확도 : 0.000

학습용 세트 정확도 : 0.000
테스트 세트 정확도 : 0.000

학습용 세트 정확도 : 0.000
테스트 세트 정확도 : 0.000



In [811]:
importance_df=pd.DataFrame(model.feature_importances_,index=X_train.columns)
importance_df.sort_values(by=0,ascending=False)

,0
총세대수,0.229401
단지내주차면수,0.211400
공가수,0.148683
지역,0.136879
label_bus,0.118798
전용면적,0.057116
자격유형,0.036027
label_generation_num,0.032671
공급유형,0.021312
임대건물구분,0.007714


### 모델만들기

In [812]:
pred=model.predict(X_test)
pred

array([ 671., 2550.,  509.,  528.,  614.,  359.,  404., 1398.,  572.,
        129.,  305.,  442., 1279.,  839.,  396.,  526., 1433.,  437.,
        606.,  682.,  296.,  588., 1077.,  246.,  528.,  713.,  359.,
        448.,  172.,  347.,  649.,  205.,  757.,  311.,  639.,  509.,
        669.,  446.,  934.,  609.,  551.,   62.,  548., 1350.,  845.,
        555.,  579.,  358.,  451.,  509.,  620.,  745.,  995.,  131.,
        745.,   94., 1214.,  246.,  744.,  460.,  735.,   94.,  333.,
        172.,  403.,  934.,  281.,  133.,  712.,  823.,   96.,  733.,
        349.,  417.,  129.,  901.,  282., 1162.,  505.,   93.,  359.,
        373.,  268.,  209., 1730.,  155.,  196.,  525.,  123., 1001.,
        548.,  951.,  751.,  254., 1044.,   65.,  444.,  172.,   78.,
        127.,  196.,  557., 1653.,  520., 1023., 1145.,  585.,   94.,
         63.,  359.,  334.,   98.,  669.,  116.,  172.,  209.,  196.,
        479.,   93.,  202., 1668.,  415.,  683.,  820.,  548.,  770.,
        123.,  373.,

In [813]:
#mae_val=sum(abs(y_test-pred))/len(pred)
mae_val=np.mean(abs(y_test-pred))
mae_val

4.308011049723757

In [814]:
#mse_val=sum((y_test-pred)**2)/len(pred)
mse_val=np.sum((y_test-pred)**2)/len(pred)
mse_val

1417.7969613259668

In [815]:
#rmse_val=mse_val ** 0.5
rmse_val=np.sqrt(mse_val)
rmse_val

37.65364472831238

### 제출파일 만들기

In [816]:
print(test.shape)
print(X_test.shape)
print(pred.shape)

(1008, 19)
(724, 10)
(724,)


In [817]:
sel=['label_generation_num','label_bus','단지내주차면수','공급유형','총세대수','전용면적','공가수','지역','임대건물구분','자격유형']
X_test=test[sel]
pred=model.predict(X_test)

In [818]:
test['등록차량수'] = pred
test['단지별차량수평균'] = test.groupby("단지코드")['등록차량수'].transform(np.mean)
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new.head()

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,label_bus,label_sub,label_generation_num,등록차량수,단지별차량수평균
0,0,C1072,754,1,1,1,39.79,116,6,8,22830000,189840,2.0,2.0,683.0,0,0,10,730.0,687.250000
1,8,C1128,1354,1,1,1,39.79,368,4,8,22830000,189840,3.0,3.0,1216.0,1,1,16,799.0,940.333333
2,17,C1456,619,1,3,1,33.40,82,9,1,19706000,156200,16.0,16.0,547.0,8,8,8,761.0,758.666667
3,26,C1840,593,1,11,1,39.57,253,3,1,14418000,108130,3.0,3.0,543.0,1,1,15,484.0,546.500000
4,30,C1332,1297,1,1,1,39.99,282,5,8,28598000,203050,2.0,2.0,1112.0,0,0,15,793.0,691.000000


In [819]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df.to_csv('0716_random_tree.csv', index=False)
sub_df.head()

,code,num
0,C1072,687.250000
1,C1128,940.333333
2,C1456,758.666667
3,C1840,546.500000
4,C1332,691.000000


In [820]:
import os
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 '0715_api_crawler.ipynb',
 '0716_dacon_form.ipynb',
 '0716_random_forest.ipynb',
 '0716_random_tree.csv',
 '0716_study.csv',
 '0716_전처리_randomforest.ipynb',
 'age_gender_info.csv',
 'cache',
 'sample_submission.csv',
 'test_df.csv',
 'train_df_errno.csv',
 'Untitled.ipynb']

### 제출용 오류 코드를 제출을 위해 3개 값을 0으로 만들고 추가

In [821]:
add_dat={'code':['C2675','C2335','C1327'],'num':['0','0','0']}
add_df=pd.DataFrame(add_dat)
add_df

,code,num
0,C2675,0
1,C2335,0
2,C1327,0


In [822]:
sub_df=pd.concat([sub_df,add_df]).reset_index()
sub_df

,index,code,num
0,0,C1072,687.25
1,1,C1128,940.333
2,2,C1456,758.667
3,3,C1840,546.5
4,4,C1332,691
...,...,...,...
145,145,C1267,370.455
146,146,C2189,189
147,0,C2675,0
148,1,C2335,0


In [823]:
sub_df=sub_df.drop(['index'],axis=1)

In [824]:
sub_df.to_csv('0716_random_tree.csv', index=False)
sub_df.head()

,code,num
0,C1072,687.25
1,C1128,940.333
2,C1456,758.667
3,C1840,546.5
4,C1332,691


In [825]:
import os
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 '0715_api_crawler.ipynb',
 '0716_dacon_form.ipynb',
 '0716_random_forest.ipynb',
 '0716_random_tree.csv',
 '0716_study.csv',
 '0716_전처리_randomforest.ipynb',
 'age_gender_info.csv',
 'cache',
 'sample_submission.csv',
 'test_df.csv',
 'train_df_errno.csv',
 'Untitled.ipynb']

In [807]:
sub_df.shape

(150, 2)